In [90]:
import pandas as pd
import numpy as np

In [92]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [94]:
df = pd.read_csv('titanic.csv',usecols=['Age','Fare','Survived'])

In [96]:
df.sample(5)

,Survived,Age,Fare
268,1,NaN,8.05
362,1,31.0,21.00
276,0,28.0,10.50
344,0,NaN,14.50
213,1,60.0,26.00


In [98]:
#drop missing row of missing values

df.dropna(inplace=True)

In [100]:
df.shape

(331, 3)

In [102]:
x = df.iloc[:,1:]
y = df.iloc[:,0]

In [104]:
# train test split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [106]:
clf = DecisionTreeClassifier()

In [108]:
clf.fit(x_train,y_train)

y_pred = clf.predict(x_test)

In [110]:
accuracy_score(y_test,y_pred)

0.582089552238806

In [112]:
np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv=10,scoring='accuracy'))

0.5589126559714794

In [114]:
kbin_age = KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')
kbin_fare = KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')

In [116]:
trf = ColumnTransformer([
    ('first',kbin_age,[0]),
    ('second',kbin_fare,[1])
])

In [118]:
x_train_trf = trf.fit_transform(x_train) 
x_test_trf = trf.transform(x_test)

In [120]:
output = pd.DataFrame({
    'age':x_train['Age'],
    'age_trf':x_train_trf[:,0],
    'fare':x_train['Fare'],
    'fare_trf':x_train_trf[:,1]
})

In [122]:
output['age_label'] = pd.cut(x=x_train['Age'],bins=trf.named_transformers_['first'].bin_edges_[0].tolist())

output['fare_label'] = pd.cut(x=x_train['Fare'],bins=trf.named_transformers_['second'].bin_edges_[0].tolist())

In [124]:
output.sample(10)

,age,age_trf,fare,fare_trf,age_label,fare_label
42,41.0,11.0,7.8500,2.0,"(39.0, 43.8]","(7.778, 7.896]"
20,55.0,14.0,59.4000,12.0,"(47.933, 55.0]","(58.41, 78.679]"
72,29.0,7.0,7.9250,3.0,"(27.733, 30.0]","(7.896, 8.662]"
376,22.0,4.0,8.6625,4.0,"(20.0, 22.0]","(7.896, 8.662]"
86,27.0,6.0,7.8792,2.0,"(25.2, 27.733]","(7.778, 7.896]"
37,21.0,3.0,8.6625,4.0,"(20.0, 22.0]","(7.896, 8.662]"
372,49.0,13.0,0.0000,0.0,"(47.933, 55.0]",NaN
213,60.0,14.0,26.0000,9.0,"(55.0, 76.0]","(21.0, 26.0]"
260,36.5,10.0,17.4000,7.0,"(36.0, 39.0]","(14.891, 21.0]"
347,38.0,10.0,7.2292,0.0,"(36.0, 39.0]","(0.0, 7.426]"


In [126]:
clf = DecisionTreeClassifier()

clf.fit(x_train_trf,y_train)
y_pred2 = clf.predict(x_test_trf) 

In [128]:
accuracy_score(y_test,y_pred2)

0.5970149253731343